In [ ]:
import numpy as np
import pandas as pd
from sklearn.utils.multiclass import unique_labels
import os
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras_preprocessing.image import ImageDataGenerator, img_to_array, load_img
import seaborn as sns
from keras import applications
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras import applications
from keras.applications import VGG19 #For Transfer Learning
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from keras.utils import to_categorical
import math
import datetime
import time
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install split-folders --quiet

In [ ]:
pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import pertinent libraries
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2
# [Keras Models]
# import the Keras implementations of VGG16, VGG19, InceptionV3 and Xception models
# the model used here is VGG16
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD
import tensorflow
from scipy.interpolate import UnivariateSpline
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import splitfolders
splitfolders.ratio("/content/drive/MyDrive/Project/Danger Of Extinction",output='Splitted',ratio=(0.8,0.1,0.1))

Copying files: 6700 files [04:14, 26.34 files/s]


In [ ]:
# [Dataset]
# image dimensions for VGG16, VGG19 are 224, 224
# image dimensions for InceptionV3 and Xception are 299, 299
img_width, img_height = 224, 224
train_dir = "./Splitted/train"
validate_dir = "./Splitted/val"
test_dir = "./Splitted/test"
nb_epochs = 10
batch_size = 32
nb_classes = len(glob.glob(train_dir + '/*'))

# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))
# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))

In [ ]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [ ]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 5349 images belonging to 21 classes.
Found 663 images belonging to 21 classes.


In [ ]:
# set up transfer learning on pre-trained ImageNet VGG19 model - remove fully connected layer and replace
# with softmax for classifying the number of classes in the dataset
vgg19_model = VGG19(weights = 'imagenet', include_top = False)
x = vgg19_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(inputs = vgg19_model.input, outputs = predictions)

80134624/80134624 [==============================] - 4s 0us/step


In [ ]:
# freeze all layers of the pre-trained InceptionV3 model
for layer in vgg19_model.layers:
    layer.trainable = False

In [ ]:
# compile the new model using a RMSProp optimizer
model.compile(optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [ ]:
# fit the model, log the results and the training time
now = datetime.datetime.now
t = now()
transfer_learning_history = model.fit_generator(
    train_generator,
    epochs = nb_epochs,
    steps_per_epoch = 100,
    validation_data = validate_generator,
    validation_steps = nb_validate_samples,
    class_weight=None)
print('Training time: %s' % (now() - t))

<ipython-input-13-f433a8d09db8>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  transfer_learning_history = model.fit_generator(


Epoch 1/10
 168/5354 [..............................] - ETA: 42:43 - loss: 1.9819 - accuracy: 0.3389

5354/5354 [==============================] - 108s 18ms/step - loss: 1.9819 - accuracy: 0.3389 - val_loss: 1.6461 - val_accuracy: 0.4585
Training time: 0:01:48.449215


In [ ]:
# evaluate the performance the new model and report the results
score = model.evaluate_generator(validate_generator, nb_validate_samples/batch_size)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

<ipython-input-15-b1de38505234>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(validate_generator, nb_validate_samples/batch_size)


Test Score: 1.6652741432189941
Test Accuracy: 0.4600301682949066


In [ ]:
from scipy import interpolate

In [ ]:
# save transfer learning model for offline prediction purposes
model.save('butterflies_vgg19_model_tl.h5')

In [ ]:
from scipy import interpolate
from scipy.interpolate import UnivariateSpline

<h1>Fine Tuning

In [ ]:
# Step 1 - Set up fine tuning on pre-trained ImageNet vgg19 model - train all layers for VGG16 and VGG19 models but only the Layers from
# 94 and above for the Inception V3 and Xception models
for layer in model.layers:
    layer.trainable = True

# Step 2 - Compile the revised model using SGD optimizer with a learing rate of 0.0001 and a momentum of 0.9
model.compile(optimizer = SGD(lr = 0.0001, momentum=0.9),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

# Step 3 - Fit the revised model, log the results and the training time
now = datetime.datetime.now
t = now()
fine_tuning_history = model.fit_generator(
    train_generator,
    epochs = nb_epochs,
    steps_per_epoch = 100,
    validation_data = validate_generator,
    validation_steps = nb_validate_samples,
    class_weight=None)
print('Training time: %s' % (now() - t))

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)
<ipython-input-24-2818f39817c9>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fine_tuning_history = model.fit_generator(


Epoch 1/10
 168/5354 [..............................] - ETA: 54:25 - loss: 1.1527 - accuracy: 0.6394

5354/5354 [==============================] - 122s 22ms/step - loss: 1.1527 - accuracy: 0.6394 - val_loss: 0.9033 - val_accuracy: 0.7255
Training time: 0:02:23.174176


In [ ]:
# evaluate the performance the new model and report the results
score = model.evaluate_generator(validate_generator, nb_validate_samples/batch_size)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

<ipython-input-25-b1de38505234>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(validate_generator, nb_validate_samples/batch_size)


Test Score: 0.9041362404823303
Test Accuracy: 0.7496229410171509


In [ ]:
# save fine-tuning model for offline prediction purposes
model.save('butterflies_vgg19_model_ft.h5')

In [ ]:
num_images = len(glob.glob("/content/drive/MyDrive/Project/download.jpg"))
predict_files = glob.glob("/content/drive/MyDrive/Project/download.jpg")

im = cv2.imread(predict_files[0])
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (256, 256)).astype(np.float32)
im = np.expand_dims(im, axis = 0)/255

predictor, image_id = [], []
for i in predict_files:
    im = cv2.imread(i)
    im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (256, 256)).astype(np.float32) / 255.0
    im = np.expand_dims(im, axis =0)
    outcome = [np.argmax(model.predict(im))]
    predictor.extend(list(outcome))
    image_id.extend([i.rsplit("\\")[-1]])

final = pd.DataFrame()
final["id"] = image_id
final["Cheetah"] = predictor

classes = train_generator.class_indices
classes = {value : key for key, value in classes.items()}

final["Cheetah"] = final["Cheetah"].apply(lambda x: classes[x])
final.head(num_images)

1/1 [==============================] - 1s 685ms/step


,id,Cheetah
0,/content/drive/MyDrive/Project/download.jpg,cheetahs
